# P1 Edgar data client for REST API

## Initialization

In [1]:
import p1_data_client_python.edgar_client as p1_edg

# Enter your token here.
# You can get your token by signing up at `www.particle.one`.
#TOKEN = "YOUR_TOKEN_HERE"
# An example token is like:
TOKEN = "8c9c9458b145202c7a6b6cceaabd82023e957a46d6cf7061ed8e1c94a168f2fd"

API_URL = 'https://data.dev.alpha.service.particle.one/edgar/v1'

## Quick start

There are 3 steps: 
1. Get information about ticker identifiers 
2. Get information about financial items available
3. Download data

## Mappers

### GvkeyCikMapper

It handles CIK <-> GVKEY transformation.

In [2]:
gvkey_mapper = p1_edg.GvkeyCikMapper(token=TOKEN, base_url=API_URL)
gvkey_mapper.get_gvkey_from_cik(cik='0000940800', as_of_date='2007-01-18')

,cik,gvkey,effdate,thrudate
0,0000940800,061411,2007-01-18T00:00:00,2007-03-14T23:59:59


In [3]:
gvkey_mapper.get_cik_from_gvkey(gvkey='061411', as_of_date='2007-01-18')

,cik,gvkey,effdate,thrudate
0,0000940800,061411,2007-01-18T00:00:00,2007-03-14T23:59:59


### CompustatItemMapper

It provides mapping between keywords and description of Compustat items.

In [4]:
item_mapper = p1_edg.CompustatItemMapper(token=TOKEN,
                                         base_url=API_URL)
item_mapper.get_item_from_keywords(
            keywords=['short-term', 'short term'])

,item,description
0,IVSTCHY,Short-Term Investments - Change
1,CHEQ,Cash and Short-Term Investments
2,ALTOQ,Other Long-term Assets
3,IVLTQ,Total Long-term Investments
4,DLTTQ,Long-Term Debt - Total
5,DLTRY,Long-Term Debt - Reduction
6,DLTISY,Long-Term Debt - Issuance


In [5]:
item_mapper.get_mapping()

,item,description
0,ACCHGQ,Accounting Changes / Cumulative Effect
1,ACOMINCQ,Accumulated Other Comprehensive Income (Loss)
2,ACOQ,Current Assets - Other - Total
3,ACTQ,Current Assets - Total
4,ADRRQ,ADR Ratio
...,...,...
167,XIDOQ,Extraordinary Items and Discontinued Operations
168,XINTQ,Interest and Related Expense- Total
169,XIQ,Extraordinary Items
170,XRDQ,Research and Development Expense


## Payload data

After collecting all necessary identifiers we are ready to download the payload data.
- form_name: Edgar form short code
- cik: Company Identification Key 
- start_date and end_date: limit our selection by dates
- items: list of items that we want to see in a result set.

Keep in mind that if you specify a broad range of search, the result could be very large.

In [6]:
client = p1_edg.EdgarClient(token=TOKEN, base_url=API_URL)

client.get_payload(form_name='8-K',
                   cik=1002910,
                   start_date='2020-11-03',
                   end_date='2020-11-04',
                   item='OIBDPQ'
                   )

,url,cik,filing_date,internal_timestamp,item,table_row_name,extracted_value,period
0,https://www.sec.gov/Archives/edgar/data/100291...,1002910,2020-11-04,1.604528e+09,OIBDPQ,Total operating revenues,1628.00,2020-11-04T00:00:00
1,https://www.sec.gov/Archives/edgar/data/100291...,1002910,2020-11-04,1.604528e+09,OIBDPQ,Other operations and maintenance,418.00,2020-11-04T00:00:00
2,https://www.sec.gov/Archives/edgar/data/100291...,1002910,2020-11-04,1.604528e+09,OIBDPQ,Taxes other than income taxes,128.00,2020-11-04T00:00:00
3,https://www.sec.gov/Archives/edgar/data/100291...,1002910,2020-11-04,1.604528e+09,OIBDPQ,Total operating expenses,1134.00,2020-11-04T00:00:00
4,https://www.sec.gov/Archives/edgar/data/100291...,1002910,2020-11-04,1.604528e+09,OIBDPQ,Operating Income,494.00,2020-11-04T00:00:00
5,https://www.sec.gov/Archives/edgar/data/100291...,1002910,2020-11-04,1.604528e+09,OIBDPQ,"Other Income, Net",48.00,2020-11-04T00:00:00
6,https://www.sec.gov/Archives/edgar/data/100291...,1002910,2020-11-04,1.604528e+09,OIBDPQ,Income Before Income Taxes,432.00,2020-11-04T00:00:00
7,https://www.sec.gov/Archives/edgar/data/100291...,1002910,2020-11-04,1.604528e+09,OIBDPQ,Income Taxes,63.00,2020-11-04T00:00:00
8,https://www.sec.gov/Archives/edgar/data/100291...,1002910,2020-11-04,1.604528e+09,OIBDPQ,Net Income,369.00,2020-11-04T00:00:00
9,https://www.sec.gov/Archives/edgar/data/100291...,1002910,2020-11-04,1.604528e+09,OIBDPQ,Less: Net Income Attributable to Noncontrollin...,2.00,2020-11-04T00:00:00
